# Check Conda Env Name

In [1]:
import sys
sys.executable.split('/')[-3]

# Imports

In [2]:
import cv2
import os
import matplotlib.pyplot as plt
import glob
import time
import gc
import pandas as pd
import numpy as np
import boto3
from PIL import Image
import io
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import ImageDraw, ImageFont
from glob import glob
from pprint import pprint
import pprint
from difflib import SequenceMatcher



# Initialization

In [3]:
BUCKET = "amazon-rekognition"
KEY = "search.jpg"
COLLECTION = "my-collection-id"

ACCESS_ID = ''
ACCESS_KEY = ''

client=boto3.client('rekognition', region_name='ap-northeast-2', aws_access_key_id=ACCESS_ID, aws_secret_access_key=ACCESS_KEY)
# rekognition = boto3.client('rekognition')

# img_subtraction_folder = '/media/data2/dataset/DF/Fake_Localized'

# if not os.path.exists(img_subtraction_folder):
#     os.mkdir(img_subtraction_folder)

# Methods

In [4]:
def compare_faces(sourceFile, targetFile):
    client = boto3.client('rekognition', region_name='us-east-1', aws_access_key_id='', aws_secret_access_key='')

    imageSource = open(sourceFile, 'rb')
    imageTarget = open(targetFile, 'rb')
    f = open('face_similarity_celebDFv2_all.csv', 'a', encoding='utf-8', newline='')
    wr = csv.writer(f)
    try:
        response = client.compare_faces(SimilarityThreshold=0,
                                    SourceImage={'Bytes': imageSource.read()},
                                    TargetImage={'Bytes': imageTarget.read()})
        for faceMatch in response['FaceMatches']:
            position = faceMatch['Face']['BoundingBox']
            similarity = str(faceMatch['Similarity'])
            wr.writerow([imageSource, imageTarget, similarity])
            print('The face at ' +
                  str(position['Left']) + ' ' +
                  str(position['Top']) +
                  'matches with ' + similarity + '% confidence')
            return len(response['FaceMatches'])
    except:
        error = "can't detect"
        wr.writerow([imageSource, imageTarget, error])
        return error
    f.close()
    imageSource.close()
    imageTarget.close()
def recognize_celebrities(photo):

    with open(photo, 'rb') as image:
        response = client.recognize_celebrities(Image={'Bytes': image.read()})

    print('Detected faces for ' + photo)
    image = Image.open(photo)
    imgWidth,imgHeight  = image.size
    draw = ImageDraw.Draw(image)
#     font = ImageFont.truetype(font=None, size=12, index=0, encoding='')
    for celebrity in response['CelebrityFaces']:
#         print ('Name: ' + celebrity['Name'])
#         print ('Confidence:',celebrity['MatchConfidence'])#celebrity.keys())
#         print ('Id: ' + celebrity['Id'])
#         print ('Position:')
#         print ('   Left: ' + '{:.2f}'.format(celebrity['Face']['BoundingBox']['Height']))
#         print ('   Top: ' + '{:.2f}'.format(celebrity['Face']['BoundingBox']['Top']))
#         print ('Info')
#         for url in celebrity['Urls']:
#             print ('   ' + url)
#         print()
        
        
        box = celebrity['Face']['BoundingBox']
        left = imgWidth * box['Left']
        top = imgHeight * box['Top']
        width = imgWidth * box['Width']
        height = imgHeight * box['Height']
        points = (
                    (left,top),
                    (left + width, top),
                    (left + width, top + height),
                    (left , top + height),
                    (left, top)
        )
        draw.line(points, fill='#00d400', width=2)
        fnt = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf",15)

        draw.text((left+(width/10),top), str(celebrity['MatchConfidence'])+":"+celebrity['Name'],fill='#FFFFFF', align ="left",font=fnt)
#     display(image)
#     print(response)
    for celebrity in response['UnrecognizedFaces']:    
        box = celebrity['BoundingBox']
        left = imgWidth * box['Left']
        top = imgHeight * box['Top']
        width = imgWidth * box['Width']
        height = imgHeight * box['Height']
        points = (
                    (left,top),
                    (left + width, top),
                    (left + width, top + height),
                    (left , top + height),
                    (left, top)
        )
        draw.line(points, fill='#d40000', width=2)
#     display(image)
    return len(response['CelebrityFaces']),response,image
def get_celebrity_dic(path):
    dic=dict()
    with open(path,"r") as csv_file:
        for line in csv_file:
            cols=line.split(",")
            dic[cols[0]]=cols[1].rstrip()
    
    # for key, value in dic.items():
    #         print(key,value)
    return dic
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
def check_case(a,b,c):
    if similar(a,c)>0.90:
        return "case 1" #result is source
    elif similar(b,c)>0.90:
        return "case 2" #result is target
    else:
        return "case 3" #result is someone-else
    
def check_case_dfP(a,b):
    if similar(a,c)>0.95:
        return "case 1" #result is source
    else:
        return "case 2" #result is someone-else
def make_output_line(first,second,src_celeb_name,tar_celeb_name):
    if first!="" and second !="":
        first_str=str(first['Name']+","+str(first['MatchConfidence']))
        second_str=str(second['Name']+","+str(second['MatchConfidence']))
        case=check_case(first['Name'],second['Name'],tar_celeb_name)
        line="{},{},{},{},{},{},{}".format(str(src_celeb_name),
                                           str(tar_celeb_name),
                                           first['Name'],
                                           str(first['MatchConfidence']),
                                           second['Name'],
                                           str(second['MatchConfidence']),
                                           case)
        return line
    elif first!="" and second =="":
        case="case 4"#result is no-one
        line="{},{},{},{},{},{},{}".format(str(src_celeb_name),
                                           str(tar_celeb_name),
                                           first['Name'],
                                           str(first['MatchConfidence']),
                                           "",
                                           "",
                                           case)
        return line
    elif first=="" and second !="":
        case=check_case("",second['Name'],tar_celeb_name)+"_1"
        line="{},{},{},{},{},{},{}".format(str(src_celeb_name),
                                           str(tar_celeb_name),
                                           "",
                                           "",
                                           second['Name'],
                                           str(second['MatchConfidence']),
                                           case)
        return line
    else:
        case="case 5"# failed to find src and target
        line="{},{},{},{},{},{},{}".format(str(src_celeb_name),
                                           str(tar_celeb_name),
                                           "",
                                           "",
                                           "",
                                           "",
                                           case)
    return line
    

# Vox2

In [6]:

dirs=glob("first-order-model/first-order-motion-model/vox2_ft_32_plots/*")
celeb_dic=get_celebrity_dic("vox_id_utf-8.csv") 
# pprint(dirs)
with open("first-order-model/first-order-motion-model/vox2_ft_32_plots_result/results.csv","w",buffering=1) as csv_file:
    csv_file.write("Filename,Real, Fake, Real Prediction, Real Prediction Confidence, Fake Prediction,Fake Prediction Confidence,Case\n") 
    for dir in dirs:
    #     print(dir)
        actor_name=os.path.basename(dir)
        print(actor_name)
        files=(glob(os.path.join(dir,"*")))
        for file in files:   
    #         pprint(files)
            file_name=os.path.basename(file)
    #         print(file_name)
            celeb_count,response,image=recognize_celebrities(file)
            print("Celebrities detected: " + str(celeb_count))
            output_s = pprint.pformat(response)
            output_dir=os.path.join("first-order-model/first-order-motion-model/","vox2_ft_32_plots_result",actor_name)
            output_filename=os.path.join(output_dir,file_name)
            output_log_filename=os.path.join(output_dir,"log_"+os.path.splitext(file_name)[0]+".txt")
    #         print(output_dir)
    #         print(output_filename)
    #         print(output_log_filename)
            if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
            open(output_log_filename, 'w').write(output_s)
            image.save(output_filename)
            real_top=99999
            real_left=99999
            real=99999
            imgWidth,imgHeight  = image.size
            lines=[]
            first=""
            second=""
            third=""
            fourth=""
#             others=[]
            for celebrity in response['CelebrityFaces']:
                box = celebrity['Face']['BoundingBox']
                left = imgWidth * box['Left']
                top = imgHeight * box['Top']
                width = imgWidth * box['Width']
                height = imgHeight * box['Height']
                bot = top + height
                right = left + width
                center_h=top + (height/2)
                center_w=left + (width/2)
#                 if top<(imgHeight/2) and left<(imgWidth/2):#first quadrant
#                     first=celebrity#str(celebrity['Name']+","+str(celebrity['MatchConfidence']))
#                 else:
#                     others.append(celebrity)
                    
                if center_h<(imgHeight/2) and center_w<(imgWidth/2):#first quadrant
                    first=celebrity
                if center_h<(imgHeight/2) and center_w>(imgWidth/2):#second quadrant
                    second=celebrity#str(","+celebrity['Name']+","+str(celebrity['MatchConfidence']))
                if center_h>(imgHeight/2) and center_w<(imgWidth/2):#third quadrant
                    third=celebrity#str(","+celebrity['Name']+","+str(celebrity['MatchConfidence']))
                if center_h>(imgHeight/2) and center_w>(imgWidth/2):#fourth quadrant
                    fourth=celebrity#str(","+celebrity['Name']+","+str(celebrity['MatchConfidence']))   


            ids=file_name.split(".")[0].split("_")[1:]
            real_id=ids[0]
            fake_ids=ids[1:]

    #         print(celeb_dic[real_id],celeb_dic[fake_ids[0]])
#             for ii,other in enumerate(fake_ids):
                
#                 csv_file.write(str(file+","+make_output_line(first,other,celeb_dic[real_id],celeb_dic[fake_ids[ii]])+"\n"))
                
            if len(fake_ids)>0:
                csv_file.write(str(file+","+make_output_line(first,second,celeb_dic[real_id],celeb_dic[fake_ids[0]])+"\n"))
            if len(fake_ids) > 1:
                csv_file.write(str(file+","+make_output_line(first,third,celeb_dic[real_id],celeb_dic[fake_ids[1]])+"\n"))
            if len(fake_ids) > 2:
                csv_file.write(str(file+","+make_output_line(first,fourth,celeb_dic[real_id],celeb_dic[fake_ids[2]])+"\n"))
#             print(make_output_line(first,second,celeb_dic[real_id],celeb_dic[fake_ids[0]]))
#             print(make_output_line(first,third,celeb_dic[real_id],celeb_dic[fake_ids[1]]))
#             print(make_output_line(first,fourth,celeb_dic[real_id],celeb_dic[fake_ids[2]]))

#             print()
            print("-_-_-_-_-_-_-_-")
    #         break
#         break



# Vox 1

In [7]:

dirs=glob("first-order-model/first-order-motion-model/vox1_32_plots/*")
celeb_dic=get_celebrity_dic("vox1_id.csv") 
# pprint(dirs)
with open("first-order-model/first-order-motion-model/vox1_32_plots_result/results.csv","w",buffering=1) as csv_file:
    csv_file.write("Filename,Real, Fake, Real Prediction, Real Prediction Confidence, Fake Prediction,Fake Prediction Confidence,Case\n") 
    for dir in dirs:
    #     print(dir)
        actor_name=os.path.basename(dir)
        print(actor_name)
        files=(glob(os.path.join(dir,"*")))
        for file in files:   
    #         pprint(files)
            file_name=os.path.basename(file)
    #         print(file_name)
            celeb_count,response,image=recognize_celebrities(file)
            print("Celebrities detected: " + str(celeb_count))
            output_s = pprint.pformat(response)
            output_dir=os.path.join("first-order-model/first-order-motion-model/","vox1_32_plots_result",actor_name)
            output_filename=os.path.join(output_dir,file_name)
            output_log_filename=os.path.join(output_dir,"log_"+os.path.splitext(file_name)[0]+".txt")
    #         print(output_dir)
    #         print(output_filename)
    #         print(output_log_filename)
            if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
            open(output_log_filename, 'w').write(output_s)
            image.save(output_filename)
            real_top=99999
            real_left=99999
            real=99999
            imgWidth,imgHeight  = image.size
            lines=[]
            first=""
            second=""
            third=""
            fourth=""
#             others=[]
            for celebrity in response['CelebrityFaces']:
                box = celebrity['Face']['BoundingBox']
                left = imgWidth * box['Left']
                top = imgHeight * box['Top']
                width = imgWidth * box['Width']
                height = imgHeight * box['Height']
                bot = top + height
                right = left + width
                center_h=top + (height/2)
                center_w=left + (width/2)
#                 if top<(imgHeight/2) and left<(imgWidth/2):#first quadrant
#                     first=celebrity#str(celebrity['Name']+","+str(celebrity['MatchConfidence']))
#                 else:
#                     others.append(celebrity)
                    
                if center_h<(imgHeight/2) and center_w<(imgWidth/2):#first quadrant
                    first=celebrity
                if center_h<(imgHeight/2) and center_w>(imgWidth/2):#second quadrant
                    second=celebrity#str(","+celebrity['Name']+","+str(celebrity['MatchConfidence']))
                if center_h>(imgHeight/2) and center_w<(imgWidth/2):#third quadrant
                    third=celebrity#str(","+celebrity['Name']+","+str(celebrity['MatchConfidence']))
                if center_h>(imgHeight/2) and center_w>(imgWidth/2):#fourth quadrant
                    fourth=celebrity#str(","+celebrity['Name']+","+str(celebrity['MatchConfidence']))   


            ids=file_name.split(".")[0].split("_")[1:]
            real_id=ids[0]
            fake_ids=ids[1:]

    #         print(celeb_dic[real_id],celeb_dic[fake_ids[0]])
#             for ii,other in enumerate(fake_ids):
                
#                 csv_file.write(str(file+","+make_output_line(first,other,celeb_dic[real_id],celeb_dic[fake_ids[ii]])+"\n"))
                
            if len(fake_ids)>0:
                csv_file.write(str(file+","+make_output_line(first,second,celeb_dic[real_id],celeb_dic[fake_ids[0]])+"\n"))
            if len(fake_ids) > 1:
                csv_file.write(str(file+","+make_output_line(first,third,celeb_dic[real_id],celeb_dic[fake_ids[1]])+"\n"))
            if len(fake_ids) > 2:
                csv_file.write(str(file+","+make_output_line(first,fourth,celeb_dic[real_id],celeb_dic[fake_ids[2]])+"\n"))
#             print(make_output_line(first,second,celeb_dic[real_id],celeb_dic[fake_ids[0]]))
#             print(make_output_line(first,third,celeb_dic[real_id],celeb_dic[fake_ids[1]]))
#             print(make_output_line(first,fourth,celeb_dic[real_id],celeb_dic[fake_ids[2]]))

#             print()
            print("-_-_-_-_-_-_-_-")
    #         break
#         break



# FOM

In [8]:
dirs=glob("first-order-model/first-order-motion-model/FOM_deepfake_plots/*")
celeb_dic=get_celebrity_dic("celebrity_id.csv") 
# pprint(dirs)
with open("first-order-model/first-order-motion-model/FOM_deepfake_plots_result/results.csv","w",buffering=1) as csv_file:
    csv_file.write("Filename,Real, Fake, Real Prediction, Real Prediction Confidence, Fake Prediction,Fake Prediction Confidence,Case\n") 
    for dir in dirs:
    #     print(dir)
        actor_name=os.path.basename(dir)
        print(actor_name)
        files=(glob(os.path.join(dir,"*")))
        for file in files:   
    #         pprint(files)
            file_name=os.path.basename(file)
    #         print(file_name)
            celeb_count,response,image=recognize_celebrities(file)
            print("Celebrities detected: " + str(celeb_count))
            output_s = pprint.pformat(response)
            output_dir=os.path.join("first-order-model/first-order-motion-model/","FOM_deepfake_plots_result",actor_name)
            output_filename=os.path.join(output_dir,file_name)
            output_log_filename=os.path.join(output_dir,"log_"+os.path.splitext(file_name)[0]+".txt")
    #         print(output_dir)
    #         print(output_filename)
    #         print(output_log_filename)
            if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
            open(output_log_filename, 'w').write(output_s)
            image.save(output_filename)
            real_top=99999
            real_left=99999
            real=99999
            imgWidth,imgHeight  = image.size
            lines=[]
            first=""
            second=""
            third=""
            fourth=""
#             others=[]
            for celebrity in response['CelebrityFaces']:
                box = celebrity['Face']['BoundingBox']
                left = imgWidth * box['Left']
                top = imgHeight * box['Top']
                width = imgWidth * box['Width']
                height = imgHeight * box['Height']
                bot = top + height
                right = left + width
                center_h=top + (height/2)
                center_w=left + (width/2)
#                 if top<(imgHeight/2) and left<(imgWidth/2):#first quadrant
#                     first=celebrity#str(celebrity['Name']+","+str(celebrity['MatchConfidence']))
#                 else:
#                     others.append(celebrity)
                    
                if center_h<(imgHeight/2) and center_w<(imgWidth/2):#first quadrant
                    first=celebrity
                if center_h<(imgHeight/2) and center_w>(imgWidth/2):#second quadrant
                    second=celebrity#str(","+celebrity['Name']+","+str(celebrity['MatchConfidence']))
                if center_h>(imgHeight/2) and center_w<(imgWidth/2):#third quadrant
                    third=celebrity#str(","+celebrity['Name']+","+str(celebrity['MatchConfidence']))
                if center_h>(imgHeight/2) and center_w>(imgWidth/2):#fourth quadrant
                    fourth=celebrity#str(","+celebrity['Name']+","+str(celebrity['MatchConfidence']))   


            ids=file_name.split(".")[0].split("_")[1:]
            real_id=ids[0]
            fake_ids=ids[1:]

    #         print(celeb_dic[real_id],celeb_dic[fake_ids[0]])
#             for ii,other in enumerate(fake_ids):
                
#                 csv_file.write(str(file+","+make_output_line(first,other,celeb_dic[real_id],celeb_dic[fake_ids[ii]])+"\n"))
                
            if len(fake_ids)>0:
                csv_file.write(str(file+","+make_output_line(first,second,celeb_dic[real_id],celeb_dic[fake_ids[0]])+"\n"))
            if len(fake_ids) > 1:
                csv_file.write(str(file+","+make_output_line(first,third,celeb_dic[real_id],celeb_dic[fake_ids[1]])+"\n"))
            if len(fake_ids) > 2:
                csv_file.write(str(file+","+make_output_line(first,fourth,celeb_dic[real_id],celeb_dic[fake_ids[2]])+"\n"))
#             print(make_output_line(first,second,celeb_dic[real_id],celeb_dic[fake_ids[0]]))
#             print(make_output_line(first,third,celeb_dic[real_id],celeb_dic[fake_ids[1]]))
#             print(make_output_line(first,fourth,celeb_dic[real_id],celeb_dic[fake_ids[2]]))

#             print()
            print("-_-_-_-_-_-_-_-")
    #         break
#         break


# CelebDF v2

In [9]:
dirs=glob("first-order-model/first-order-motion-model/images_names/*")
celeb_dic=get_celebrity_dic("celebrity_id.csv") 
# pprint(dirs)
with open("first-order-model/first-order-motion-model/images_names_result/results.csv","w",buffering=1) as csv_file:
    csv_file.write("Filename,Real, Fake, Real Prediction, Real Prediction Confidence, Fake Prediction,Fake Prediction Confidence,Case\n") 
    for dir in dirs:
    #     print(dir)
        actor_name=os.path.basename(dir)
        print(actor_name)
        files=(glob(os.path.join(dir,"*")))
        for file in files:   
    #         pprint(files)
            file_name=os.path.basename(file)
    #         print(file_name)
            celeb_count,response,image=recognize_celebrities(file)
            print("Celebrities detected: " + str(celeb_count))
            output_s = pprint.pformat(response)
            output_dir=os.path.join("first-order-model/first-order-motion-model/","images_names_result",actor_name)
            output_filename=os.path.join(output_dir,file_name)
            output_log_filename=os.path.join(output_dir,"log_"+os.path.splitext(file_name)[0]+".txt")
    #         print(output_dir)
    #         print(output_filename)
    #         print(output_log_filename)
            if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
            open(output_log_filename, 'w').write(output_s)
            image.save(output_filename)
            real_top=99999
            real_left=99999
            real=99999
            imgWidth,imgHeight  = image.size
            lines=[]
            first=""
            second=""
            third=""
            fourth=""
#             others=[]
            for celebrity in response['CelebrityFaces']:
                box = celebrity['Face']['BoundingBox']
                left = imgWidth * box['Left']
                top = imgHeight * box['Top']
                width = imgWidth * box['Width']
                height = imgHeight * box['Height']
                bot = top + height
                right = left + width
                center_h=top + (height/2)
                center_w=left + (width/2)
#                 if top<(imgHeight/2) and left<(imgWidth/2):#first quadrant
#                     first=celebrity#str(celebrity['Name']+","+str(celebrity['MatchConfidence']))
#                 else:
#                     others.append(celebrity)
                    
                if center_h<(imgHeight/2) and center_w<(imgWidth/2):#first quadrant
                    first=celebrity
                if center_h<(imgHeight/2) and center_w>(imgWidth/2):#second quadrant
                    second=celebrity#str(","+celebrity['Name']+","+str(celebrity['MatchConfidence']))
                if center_h>(imgHeight/2) and center_w<(imgWidth/2):#third quadrant
                    third=celebrity#str(","+celebrity['Name']+","+str(celebrity['MatchConfidence']))
                if center_h>(imgHeight/2) and center_w>(imgWidth/2):#fourth quadrant
                    fourth=celebrity#str(","+celebrity['Name']+","+str(celebrity['MatchConfidence']))   


            ids=file_name.split(".")[0].split("_")[1:]
            real_id=ids[0]
            fake_ids=ids[1:]

    #         print(celeb_dic[real_id],celeb_dic[fake_ids[0]])
#             for ii,other in enumerate(fake_ids):
                
#                 csv_file.write(str(file+","+make_output_line(first,other,celeb_dic[real_id],celeb_dic[fake_ids[ii]])+"\n"))
                
            if len(fake_ids)>0:
                csv_file.write(str(file+","+make_output_line(first,second,celeb_dic[real_id],celeb_dic[fake_ids[0]])+"\n"))
            if len(fake_ids) > 1:
                csv_file.write(str(file+","+make_output_line(first,third,celeb_dic[real_id],celeb_dic[fake_ids[1]])+"\n"))
            if len(fake_ids) > 2:
                csv_file.write(str(file+","+make_output_line(first,fourth,celeb_dic[real_id],celeb_dic[fake_ids[2]])+"\n"))
#             print(make_output_line(first,second,celeb_dic[real_id],celeb_dic[fake_ids[0]]))
#             print(make_output_line(first,third,celeb_dic[real_id],celeb_dic[fake_ids[1]]))
#             print(make_output_line(first,fourth,celeb_dic[real_id],celeb_dic[fake_ids[2]]))

#             print()
            print("-_-_-_-_-_-_-_-")
    #         break
#         break


# DFP

In [10]:
dirs=glob("first-order-model/first-order-motion-model/DFP_plots/*")
celeb_dic=get_celebrity_dic("dfp_id.csv") 
# pprint(dirs)
with open("first-order-model/first-order-motion-model/DFP_plots_result/results.csv","w",buffering=1) as csv_file:
    csv_file.write("Filename,Real, Fake, Real Prediction, Real Prediction Confidence, Fake Prediction,Fake Prediction Confidence,Case\n") 
    for dir in dirs:
    #     print(dir)
        actor_name=os.path.basename(dir)
        print(actor_name)
        files=(glob(os.path.join(dir,"*")))
        for file in files:   
    #         pprint(files)
            file_name=os.path.basename(file)
    #         print(file_name)
            celeb_count,response,image=recognize_celebrities(file)
            print("Celebrities detected: " + str(celeb_count))
            output_s = pprint.pformat(response)
            output_dir=os.path.join("first-order-model/first-order-motion-model/","DFP_plots_result",actor_name)
            output_filename=os.path.join(output_dir,file_name)
            output_log_filename=os.path.join(output_dir,"log_"+os.path.splitext(file_name)[0]+".txt")
    #         print(output_dir)
    #         print(output_filename)
    #         print(output_log_filename)
            if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
            open(output_log_filename, 'w').write(output_s)
            image.save(output_filename)
            real_top=99999
            real_left=99999
            real=99999
            imgWidth,imgHeight  = image.size
            lines=[]
            first=""
            second=""
            third=""
            fourth=""
#             others=[]
            for celebrity in response['CelebrityFaces']:
                box = celebrity['Face']['BoundingBox']
                left = imgWidth * box['Left']
                top = imgHeight * box['Top']
                width = imgWidth * box['Width']
                height = imgHeight * box['Height']
                bot = top + height
                right = left + width
                center_h=top + (height/2)
                center_w=left + (width/2)
#                 if top<(imgHeight/2) and left<(imgWidth/2):#first quadrant
#                     first=celebrity#str(celebrity['Name']+","+str(celebrity['MatchConfidence']))
#                 else:
#                     others.append(celebrity)
                    
                if center_h<(imgHeight/2) and center_w<(imgWidth/2):#first quadrant
                    first=celebrity
                if center_h<(imgHeight/2) and center_w>(imgWidth/2):#second quadrant
                    second=celebrity#str(","+celebrity['Name']+","+str(celebrity['MatchConfidence']))
                if center_h>(imgHeight/2) and center_w<(imgWidth/2):#third quadrant
                    third=celebrity#str(","+celebrity['Name']+","+str(celebrity['MatchConfidence']))
                if center_h>(imgHeight/2) and center_w>(imgWidth/2):#fourth quadrant
                    fourth=celebrity#str(","+celebrity['Name']+","+str(celebrity['MatchConfidence']))   


            ids=file_name.split(".")[0].split("_")[1:]
            real_id=ids[0]
            fake_ids=ids[1:]

    #         print(celeb_dic[real_id],celeb_dic[fake_ids[0]])
#             for ii,other in enumerate(fake_ids):
                
#                 csv_file.write(str(file+","+make_output_line(first,other,celeb_dic[real_id],celeb_dic[fake_ids[ii]])+"\n"))
                
            if len(fake_ids)>0:
                csv_file.write(str(file+","+make_output_line(first,second,celeb_dic[real_id],celeb_dic[fake_ids[0]])+"\n"))
            if len(fake_ids) > 1:
                csv_file.write(str(file+","+make_output_line(first,third,celeb_dic[real_id],celeb_dic[fake_ids[1]])+"\n"))
            if len(fake_ids) > 2:
                csv_file.write(str(file+","+make_output_line(first,fourth,celeb_dic[real_id],celeb_dic[fake_ids[2]])+"\n"))
#             print(make_output_line(first,second,celeb_dic[real_id],celeb_dic[fake_ids[0]]))
#             print(make_output_line(first,third,celeb_dic[real_id],celeb_dic[fake_ids[1]]))
#             print(make_output_line(first,fourth,celeb_dic[real_id],celeb_dic[fake_ids[2]]))

#             print()
            print("-_-_-_-_-_-_-_-")
    #         break
#         break


# Celeb Morphing

In [37]:
# ids=file_name.split(".")[0].split("_")[1:]
# real_id=ids[0]
# fake_ids=ids[1:]
# print(real_id,fake_id)
# for fake_id in fake_ids:
#     print(celeb_dic[fake_id])
import re
    
dirs=glob("first-order-model/first-order-motion-model/celebrity_morphing/*")
dirs.sort()
celeb_dic=get_celebrity_dic("vox1_id.csv") 
# pprint(dirs)
with open("first-order-model/first-order-motion-model/celebrity_morphing_result/results.csv","w",buffering=1) as csv_file:
    csv_file.write("Filename,Source, Target, Source Prediction, Source Prediction Confidence, Target Prediction,Target Prediction Confidence, Morph Prediciton, Morph Prediction Confidence\n") 
    for dir in dirs:
    #     print(dir)
        actor_name=os.path.basename(dir)
        print(actor_name)
        files=(glob(os.path.join(dir,"*")))
        files.sort(key=lambda f: int(re.sub('\D', '', f)))
        for file in files:   
    #         pprint(files)
            file_name=os.path.basename(file)
    #         print(file_name)
            celeb_count,response,image=recognize_celebrities(file)
            print("Celebrities detected: " + str(celeb_count))
            output_s = pprint.pformat(response)
            output_dir=os.path.join("first-order-model/first-order-motion-model/","celebrity_morphing_result",actor_name)
            output_filename=os.path.join(output_dir,file_name)
            output_log_filename=os.path.join(output_dir,"log_"+os.path.splitext(file_name)[0]+".txt")
    #         print(output_dir)
    #         print(output_filename)
    #         print(output_log_filename)
            if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
            open(output_log_filename, 'w').write(output_s)
            image.save(output_filename)
            real_top=99999
            real_left=99999
            real=99999
            imgWidth,imgHeight  = image.size
            lines=[]
            morph=""
            src=""
            tar=""
#             print(file_name)

            for celebrity in response['CelebrityFaces']:
                box = celebrity['Face']['BoundingBox']
                left = imgWidth * box['Left']
                top = imgHeight * box['Top']
                if top<(imgHeight/2):#top half
                    morph=celebrity
                elif top>(imgHeight/2) and left<(imgWidth/2):#bot left
                    src=celebrity#str(","+celebrity['Name']+","+str(celebrity['MatchConfidence']))
                elif top>(imgHeight/2) and left>(imgWidth/2):#bot right
                    tar=celebrity#str(","+celebrity['Name']+","+str(celebrity['MatchConfidence']))   
#             print('morph',morph)
#             print('src',src)
#             print('tar',tar)
            
            if src=="":
                src={}
                src['Name']=""                                       
                src['MatchConfidence']=""
            if tar=="":
                tar={}
                tar['Name']=""                                       
                tar['MatchConfidence']=""
            if morph=="":
                morph={}
                morph['Name']=""                                       
                morph['MatchConfidence']=""

            line="{},{},{},{},{},{},{},{},{}\n".format(file,
                                                     str(actor_name),
                                                     str(tar['Name']),                                          
                                                     str(src['Name']),                                          
                                                     str(src['MatchConfidence']),                                          
                                                     str(tar['Name']),                                          
                                                     str(tar['MatchConfidence']),                                          
                                                     str(morph['Name']),                                          
                                                     str(morph['MatchConfidence']))
            csv_file.write(line)
#             print(line)
            print("-_-_-_-_-_-_-_-")
#             break
#         break
    
#             ids=file_name.split(".")[0].split("_")[1:]
#             real_id=ids[0]
#             fake_ids=ids[1:]

    #         print(celeb_dic[real_id],celeb_dic[fake_ids[0]])
    
    
    
#             if len(fake_ids)>0:
#                 csv_file.write(str(file+","+make_output_line(first,second,celeb_dic[real_id],celeb_dic[fake_ids[0]])+"\n"))
#             if len(fake_ids) > 1:
#                 csv_file.write(str(file+","+make_output_line(first,third,celeb_dic[real_id],celeb_dic[fake_ids[1]])+"\n"))
#             if len(fake_ids) > 2:
#                 csv_file.write(str(file+","+make_output_line(first,fourth,celeb_dic[real_id],celeb_dic[fake_ids[2]])+"\n"))
                
                
                
#             print(make_output_line(first,second,celeb_dic[real_id],celeb_dic[fake_ids[0]]))
#             print(make_output_line(first,third,celeb_dic[real_id],celeb_dic[fake_ids[1]]))
#             print(make_output_line(first,fourth,celeb_dic[real_id],celeb_dic[fake_ids[2]]))

#             print()
#             print("-_-_-_-_-_-_-_-")
    #         break
#         break

    